## Idea Trabajo Práctico Final: Predicción de Resultado de Partidos de Tenis 
El objetivo de la red neuronal a implementar es poder predecir los resultados de cierta temporada de tenis actual considerando el historial de partidos de temporadas pasadas, como así también el historial individual entre los jugadores de cada partido. 
### Dataset de Referencia
El dataset de referencia será el historial de partidos de la Asociación de Tenistas Profesionales (ATP) brindado en su base de datos. Este no sólo incluye quien ha ganado cada partido, sino el puntaje y los porcentajes de las características relevantes de cada jugada en particular ('primeros servicios', 'segundos servicios', 'aces', 'porcentaje de devoluciones', 'doble faltas', etc.).
Por lo tanto, el set de entrenamiento de la red consistirá de un vector de características de entrada (X) representando las características de los jugadores involucrados y del partido, y un vector de salida correspondiente que determina que jugador gana. 
A continuación se realiza el parseo de datos:


In [1]:
import numpy as np

from numpy import genfromtxt
my_data_stats = genfromtxt('data/match_stats.csv', delimiter=',',dtype=None)
my_data_scores = genfromtxt('data/match_scores.csv', delimiter=',',dtype=None)


C:\Users\cufar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  after removing the cwd from sys.path.
C:\Users\cufar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """


In [2]:
my_data_stats

array([( 0, b'1991-7308-k181-l206', b'/en/scores/1991/7308/MS002/match-stats', b'01:20:00',  80,  3, 0,  0, 65, 29, 46, 12, 19, 0, 2, 41, 65, 12, 36, 20, 33, 4,  7, 11, 11, 32, 69,  73, 134,  1, 0, 36, 69, 24, 36, 13, 33, 3,  7, 37, 69, 17, 46, 7, 19, 2, 2, 11, 11, 24, 65, 61, 134),
       ( 0, b'1991-7308-k181-s351', b'/en/scores/1991/7308/MS001/match-stats', b'01:29:00',  89,  1, 2, 44, 65, 31, 44, 12, 21, 2, 5, 43, 65, 16, 47, 21, 36, 4,  7, 12, 12, 37, 83,  80, 148, 12, 5, 47, 83, 31, 47, 15, 36, 3,  7, 46, 83, 13, 44, 9, 21, 3, 5, 12, 12, 22, 65, 68, 148),
       ( 0, b'1991-7308-k181-s424', b'/en/scores/1991/7308/MS005/match-stats', b'00:55:00',  55,  0, 1, 17, 36, 11, 17, 13, 19, 1, 3, 24, 36, 15, 29, 19, 25, 7, 10,  7,  8, 34, 54,  58,  90,  2, 2, 29, 54, 14, 29,  6, 25, 3, 10, 20, 54,  6, 17, 6, 19, 2, 3,  8,  7, 12, 36, 32,  90),
       ...,
       (65, b'2016-605-n552-w367', b'/en/scores/2016/605/MS011/match-stats', b'01:08:00',  68,  5, 1, 20, 43, 16, 20, 17, 23, 0, 0, 33, 

In [3]:
import pandas as pd
pd_my_data_scores = pd.DataFrame(data=my_data_scores)
pd_my_data_scores.columns=['tourney_year_id','tourney_order','tourney_slug','tourney_url_suffix','tourney_round_name','round_order','match_order','winner_name','winner_player_id','winner_slug','loser_name','loser_player_id','loser_slug','winner_seed','loser_seed','match_score_tiebreaks','winner_sets_won','loser_sets_won','winner_games_won','loser_games_won','winner_tiebreaks_won','loser_tiebreaks_won','match_id','match_stats_url_suffix']

pd_my_data_stats = pd.DataFrame(data=my_data_stats)
pd_my_data_stats.columns=['ms_tourney_order','ms_match_id','ms_match_stats_url_suffix','ms_match_time','ms_match_duration','ms_winner_aces','ms_winner_double_faults','ms_winner_first_serves_in','ms_winner_first_serves_total','ms_winner_first_serve_points_won','ms_winner_first_serve_points_total','ms_winner_second_serve_points_won','ms_winner_second_serve_points_total','ms_winner_break_points_saved','ms_winner_break_points_serve_total','ms_winner_service_points_won','ms_winner_service_points_total','ms_winner_first_serve_return_won','ms_winner_first_serve_return_total','ms_winner_second_serve_return_won','ms_winner_second_serve_return_total','ms_winner_break_points_converted','ms_winner_break_points_return_total','ms_winner_service_games_played','ms_winner_return_games_played','ms_winner_return_points_won','ms_winner_return_points_total','ms_winner_total_points_won','ms_winner_total_points_total','ms_loser_aces','ms_loser_double_faults','ms_loser_first_serves_in','ms_loser_first_serves_total','ms_loser_first_serve_points_won','ms_loser_first_serve_points_total','ms_loser_second_serve_points_won','ms_loser_second_serve_points_total','ms_loser_break_points_saved','ms_loser_break_points_serve_total','ms_loser_service_points_won','ms_loser_service_points_total','ms_loser_first_serve_return_won','ms_loser_first_serve_return_total','ms_loser_second_serve_return_won','ms_loser_second_serve_return_total','ms_loser_break_points_converted','ms_loser_break_points_return_total','ms_loser_service_games_played','ms_loser_return_games_played','ms_loser_return_points_won','ms_loser_return_points_total','ms_loser_total_points_won','ms_loser_total_points_total']

Debajo se muestra un ejemplo del dataset scores obtenido:

In [4]:
pd_my_data_scores.loc[[0,1,2,3,4,5,95354,95355,95356,95357,95358]]

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
0,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Finals',1,1,b'Nicklas Kulti',b'k181',b'nicklas-kulti',...,b'6',b'63 16 62',2,1,13,11,0,0,b'1991-7308-k181-s351',b'/en/scores/1991/7308/MS001/match-stats'
1,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Semi-Finals',2,1,b'Michael Stich',b's351',b'michael-stich',...,b'2',b'64 76(6)',2,0,13,10,1,0,b'1991-7308-s351-c243',b'/en/scores/1991/7308/MS003/match-stats'
2,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Semi-Finals',2,2,b'Nicklas Kulti',b'k181',b'nicklas-kulti',...,b'',b'75 64',2,0,13,9,0,0,b'1991-7308-k181-l206',b'/en/scores/1991/7308/MS002/match-stats'
3,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Quarter-Finals',3,1,b'Jim Courier',b'c243',b'jim-courier',...,b'Q',b'76(3) 63',2,0,13,9,1,0,b'1991-7308-c243-s367',b'/en/scores/1991/7308/MS007/match-stats'
4,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Quarter-Finals',3,2,b'Michael Stich',b's351',b'michael-stich',...,b'',b'36 63 63',2,1,15,12,0,0,b'1991-7308-s351-a031',b'/en/scores/1991/7308/MS006/match-stats'
5,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Quarter-Finals',3,3,b'Nicklas Kulti',b'k181',b'nicklas-kulti',...,b'',b'63 60',2,0,12,3,0,0,b'1991-7308-k181-s424',b'/en/scores/1991/7308/MS005/match-stats'
95354,b'2016-605',67,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2016...,b'Round Robin',3,8,b'Dominic Thiem',b'tb69',b'dominic-thiem',...,b'6',b'63 16 64',2,1,13,13,0,0,b'2016-605-tb69-mc65',b'/en/scores/2016/605/MS021/match-stats'
95355,b'2016-605',67,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2016...,b'Round Robin',3,9,b'Andy Murray',b'mc10',b'andy-murray',...,b'7',b'63 62',2,0,12,5,0,0,b'2016-605-mc10-c977',b'/en/scores/2016/605/MS010/match-stats'
95356,b'2016-605',67,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2016...,b'Round Robin',3,10,b'Kei Nishikori',b'n552',b'kei-nishikori',...,b'3',b'62 63',2,0,12,5,0,0,b'2016-605-n552-w367',b'/en/scores/2016/605/MS011/match-stats'
95357,b'2016-605',67,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2016...,b'Round Robin',3,11,b'Novak Djokovic',b'd643',b'novak-djokovic',...,b'8',b'67(10) 60 62',2,1,18,9,0,1,b'2016-605-d643-tb69',b'/en/scores/2016/605/MS018/match-stats'


In [4]:
pd_my_data_stats.loc[[0,1,2,3,4,5,91951,91952,91953,91954,91955]]

,ms_tourney_order,ms_match_id,ms_match_stats_url_suffix,ms_match_time,ms_match_duration,ms_winner_aces,ms_winner_double_faults,ms_winner_first_serves_in,ms_winner_first_serves_total,ms_winner_first_serve_points_won,...,ms_loser_second_serve_return_won,ms_loser_second_serve_return_total,ms_loser_break_points_converted,ms_loser_break_points_return_total,ms_loser_service_games_played,ms_loser_return_games_played,ms_loser_return_points_won,ms_loser_return_points_total,ms_loser_total_points_won,ms_loser_total_points_total
0,0,b'1991-7308-k181-l206',b'/en/scores/1991/7308/MS002/match-stats',b'01:20:00',80,3,0,0,65,29,...,7,19,2,2,11,11,24,65,61,134
1,0,b'1991-7308-k181-s351',b'/en/scores/1991/7308/MS001/match-stats',b'01:29:00',89,1,2,44,65,31,...,9,21,3,5,12,12,22,65,68,148
2,0,b'1991-7308-k181-s424',b'/en/scores/1991/7308/MS005/match-stats',b'00:55:00',55,0,1,17,36,11,...,6,19,2,3,8,7,12,36,32,90
3,0,b'1991-7308-a031-s331',b'/en/scores/1991/7308/MS013/match-stats',b'01:09:00',69,2,3,44,58,28,...,7,14,2,8,9,9,23,58,52,114
4,0,b'1991-7308-k030-w136',b'/en/scores/1991/7308/MS009/match-stats',b'01:29:00',89,6,3,37,71,25,...,15,34,2,3,10,10,27,71,59,135
5,0,b'1991-7308-l206-z006',b'/en/scores/1991/7308/MS008/match-stats',b'01:49:00',109,10,0,0,87,40,...,10,35,0,0,12,12,22,87,89,178
91951,65,b'2016-605-tb69-mc65',b'/en/scores/2016/605/MS021/match-stats',b'01:30:00',90,4,1,47,71,34,...,12,24,3,3,13,13,25,71,74,148
91952,65,b'2016-605-mc10-c977',b'/en/scores/2016/605/MS010/match-stats',b'01:31:00',91,3,4,31,58,23,...,11,27,1,5,8,9,19,58,43,103
91953,65,b'2016-605-n552-w367',b'/en/scores/2016/605/MS011/match-stats',b'01:08:00',68,5,1,20,43,16,...,6,23,0,0,9,8,10,43,41,101
91954,65,b'2016-605-d643-tb69',b'/en/scores/2016/605/MS018/match-stats',b'02:03:00',123,3,0,54,80,39,...,6,26,0,1,13,13,21,80,75,176


In [6]:
display(pd_my_data_stats.shape)
display(pd_my_data_scores.shape)

(91956, 53)

(95359, 24)

In [8]:
pd_my_data_scores_2014.merge(pd_my_data_stats_2014)

MergeError: No common columns to perform merge on

In [9]:
pd_my_data_scores_2014=pd_my_data_scores.loc[pd_my_data_scores['match_id'].astype(str).str.contains('2014-605', regex=False) == True]
pd_my_data_stats_2014=pd_my_data_stats.loc[pd_my_data_stats['ms_match_id'].astype(str).str.contains('2014-605', regex=False) == True]
pd_my_data_scores_2014=pd_my_data_scores_2014.reset_index()
pd_my_data_stats_2014=pd_my_data_stats_2014.reset_index()
#pd_my_data_stats_2014.rename(columns={'ms_match_stats_url_suffix': 'match_stats_url_suffix'}, inplace=True)


In [10]:
pd_my_data_scores_2014

,index,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
0,86955,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Finals',1,1,b'Novak Djokovic',b'd643',...,b'2',b'(W/O)',0,0,0,0,0,0,b'2014-605-d643-f324',b''
1,86956,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Semi-Finals',2,1,b'Novak Djokovic',b'd643',...,b'4',b'61 36 60',2,1,15,7,0,0,b'2014-605-d643-n552',b'/en/scores/2014/605/MS002/match-stats'
2,86957,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Semi-Finals',2,2,b'Roger Federer',b'f324',...,b'3',b'46 75 76(6)',2,1,18,17,1,0,b'2014-605-f324-w367',b'/en/scores/2014/605/MS003/match-stats'
3,86958,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,1,b'Novak Djokovic',b'd643',...,b'6',b'62 62',2,0,12,4,0,0,b'2014-605-d643-ba47',b'/en/scores/2014/605/MS009/match-stats'
4,86959,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,2,b'Stan Wawrinka',b'w367',...,b'8',b'63 46 63',2,1,16,12,0,0,b'2014-605-w367-c977',b'/en/scores/2014/605/MS012/match-stats'
5,86960,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,3,b'Roger Federer',b'f324',...,b'5',b'60 61',2,0,12,1,0,0,b'2014-605-f324-mc10',b'/en/scores/2014/605/MS017/match-stats'
6,86961,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,4,b'Kei Nishikori',b'n552',...,b'9',b'46 64 61',2,1,16,11,0,0,b'2014-605-n552-f401',b'/en/scores/2014/605/MS020/match-stats'
7,86962,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,5,b'Novak Djokovic',b'd643',...,b'3',b'63 60',2,0,12,3,0,0,b'2014-605-d643-w367',b'/en/scores/2014/605/MS008/match-stats'
8,86963,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,6,b'Tomas Berdych',b'ba47',...,b'8',b'63 61',2,0,12,4,0,0,b'2014-605-ba47-c977',b'/en/scores/2014/605/MS013/match-stats'
9,86964,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,7,b'Roger Federer',b'f324',...,b'4',b'63 62',2,0,12,5,0,0,b'2014-605-f324-n552',b'/en/scores/2014/605/MS016/match-stats'


In [56]:
pd_my_data_stats_2014

,index,ms_tourney_order,ms_match_id,match_stats_url_suffix,ms_match_time,ms_match_duration,ms_winner_aces,ms_winner_double_faults,ms_winner_first_serves_in,ms_winner_first_serves_total,...,ms_loser_second_serve_return_won,ms_loser_second_serve_return_total,ms_loser_break_points_converted,ms_loser_break_points_return_total,ms_loser_service_games_played,ms_loser_return_games_played,ms_loser_return_points_won,ms_loser_return_points_total,ms_loser_total_points_won,ms_loser_total_points_total
0,83585,62,b'2014-605-d643-n552',b'/en/scores/2014/605/MS002/match-stats',b'01:28:00',88,3,2,34,58,...,11,24,2,4,11,11,17,58,51,125
1,83586,62,b'2014-605-f324-w367',b'/en/scores/2014/605/MS003/match-stats',b'02:48:00',168,4,0,57,108,...,22,51,3,6,17,17,38,108,112,231
2,83587,62,b'2014-605-w367-c977',b'/en/scores/2014/605/MS012/match-stats',b'01:46:00',106,7,4,38,78,...,15,40,2,3,14,14,25,78,77,164
3,83588,62,b'2014-605-n552-f401',b'/en/scores/2014/605/MS020/match-stats',b'01:57:00',117,5,6,49,84,...,20,35,2,9,14,13,30,84,80,172
4,83589,62,b'2014-605-d643-w367',b'/en/scores/2014/605/MS008/match-stats',b'01:05:00',65,3,0,34,48,...,7,14,1,3,7,8,15,48,31,93
5,83590,62,b'2014-605-f324-mc10',b'/en/scores/2014/605/MS017/match-stats',b'00:56:00',56,0,1,14,36,...,8,22,0,0,6,7,8,36,24,78
6,83591,62,b'2014-605-d643-ba47',b'/en/scores/2014/605/MS009/match-stats',b'01:09:00',69,3,1,24,41,...,5,17,0,0,8,8,9,41,34,93
7,83592,62,b'2014-605-f324-n552',b'/en/scores/2014/605/MS016/match-stats',b'01:09:00',69,7,1,34,59,...,11,25,0,3,8,9,18,59,48,111
8,83593,62,b'2014-605-mc10-r975',b'/en/scores/2014/605/MS021/match-stats',b'01:32:00',92,4,2,36,66,...,14,30,1,1,10,11,22,66,59,128
9,83594,62,b'2014-605-ba47-c977',b'/en/scores/2014/605/MS013/match-stats',b'01:15:00',75,5,1,28,51,...,7,23,0,6,8,8,15,51,43,106


In [11]:
results_2014=pd_my_data_scores_2014.merge(pd_my_data_stats_2014, left_on='match_stats_url_suffix', right_on='ms_match_stats_url_suffix', how='outer')
pd.set_option('display.max_rows', 100)
display(results_2014.loc[0])

index_x                                                                            86955
tourney_year_id                                                              b'2014-605'
tourney_order                                                                         64
tourney_slug                                                         b'nitto-atp-finals'
tourney_url_suffix                     b'/en/scores/archive/nitto-atp-finals/605/2014...
tourney_round_name                                                             b'Finals'
round_order                                                                            1
match_order                                                                            1
winner_name                                                            b'Novak Djokovic'
winner_player_id                                                                 b'd643'
winner_slug                                                            b'novak-djokovic'
loser_name           

In [23]:
results=pd_my_data_scores.merge(pd_my_data_stats, left_on='match_stats_url_suffix', right_on='ms_match_stats_url_suffix', how='outer')
pd.set_option('display.max_rows', 100)
#display(results.loc[20000])
results

results_2014=results.loc[results['match_id'].astype(str).str.contains('2014-605', regex=False) == True]
results_2014

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,ms_loser_second_serve_return_won,ms_loser_second_serve_return_total,ms_loser_break_points_converted,ms_loser_break_points_return_total,ms_loser_service_games_played,ms_loser_return_games_played,ms_loser_return_points_won,ms_loser_return_points_total,ms_loser_total_points_won,ms_loser_total_points_total
3389,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Finals',1,1,b'Novak Djokovic',b'd643',b'novak-djokovic',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87058,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Semi-Finals',2,1,b'Novak Djokovic',b'd643',b'novak-djokovic',...,11.0,24.0,2.0,4.0,11.0,11.0,17.0,58.0,51.0,125.0
87059,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Semi-Finals',2,2,b'Roger Federer',b'f324',b'roger-federer',...,22.0,51.0,3.0,6.0,17.0,17.0,38.0,108.0,112.0,231.0
87060,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,1,b'Novak Djokovic',b'd643',b'novak-djokovic',...,5.0,17.0,0.0,0.0,8.0,8.0,9.0,41.0,34.0,93.0
87061,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,2,b'Stan Wawrinka',b'w367',b'stan-wawrinka',...,15.0,40.0,2.0,3.0,14.0,14.0,25.0,78.0,77.0,164.0
87062,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,3,b'Roger Federer',b'f324',b'roger-federer',...,8.0,22.0,0.0,0.0,6.0,7.0,8.0,36.0,24.0,78.0
87063,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,4,b'Kei Nishikori',b'n552',b'kei-nishikori',...,20.0,35.0,2.0,9.0,14.0,13.0,30.0,84.0,80.0,172.0
87064,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,5,b'Novak Djokovic',b'd643',b'novak-djokovic',...,7.0,14.0,1.0,3.0,7.0,8.0,15.0,48.0,31.0,93.0
87065,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,6,b'Tomas Berdych',b'ba47',b'tomas-berdych',...,7.0,23.0,0.0,6.0,8.0,8.0,15.0,51.0,43.0,106.0
87066,b'2014-605',64,b'nitto-atp-finals',b'/en/scores/archive/nitto-atp-finals/605/2014...,b'Round Robin',3,7,b'Roger Federer',b'f324',b'roger-federer',...,11.0,25.0,0.0,3.0,8.0,9.0,18.0,59.0,48.0,111.0


In [20]:
results_2014.filter(['winner_name','loser_name'], axis=1)


,winner_name,loser_name
3389,b'Novak Djokovic',b'Roger Federer'
87058,b'Novak Djokovic',b'Kei Nishikori'
87059,b'Roger Federer',b'Stan Wawrinka'
87060,b'Novak Djokovic',b'Tomas Berdych'
87061,b'Stan Wawrinka',b'Marin Cilic'
87062,b'Roger Federer',b'Andy Murray'
87063,b'Kei Nishikori',b'David Ferrer'
87064,b'Novak Djokovic',b'Stan Wawrinka'
87065,b'Tomas Berdych',b'Marin Cilic'
87066,b'Roger Federer',b'Kei Nishikori'


Los datos luego serán seleccionados partido a partido, a partir del dataset obtenido. Se realiza la siguiente división:

| Categoría  |  Detalle | 
|---|---|
| ID1 |  ID Jugador 1 |
| ID2 |  ID Jugador 2 |
| RANK |  Ranking ATP |
| POINTS  |  Puntos ATP | 
| FS | Porcentaje  de primeros servicios |
| W1SP | Porcentaje de primeros servicios ganados |
| W2SP | Porcentaje de segundos servicios ganados |
| WSP | Porcentaje de servicios ganados globales |
| WRP | Porcentaje de devolución ganada |
| TPW | Porcentaje de puntos totales ganados |
| TMW | Porcentaje de todos los partidos ganados |
| ACES | Promedio de aces por game |
| DF | Promedio de doble faltas por game |
| UE | Promedio de errores no forzados por game |
| WIS | Promedio de winners por game |
| BP | Porcentaje de break points ganados |
| NA | Porcentaje de puntos en la red ganados |
| A1S | Promedio de velocidad primer servicio |
| A2S | Promedio de velocidad de segundo servicio |
| FATIGUE | Fatiga por partidos jugados en 3 dias anteriores |
| RETIRED | Indica si es el primer partido luego de un retiro |
| DIRECT | Ventaja H2H entre los jugadores |

Cada partido tiene estas características que suelen indicar las diferencias entre ambos jugadores. Por lo que por ejemplo, para ATPRank, si RANK1 es el ranking del jugador 1 y RANK2 es el ranking del jugador 2, RANK = RANK1 - RANK2.
Para el entrenamiento, cada partido tendrá como salida un vector que inidque que jugador gana o que jugador pierde. Es decir, 1 si el jugador ganó o 0 si el jugador perdió.

Finalmente, el dataset será dividido en tres partes:
- Set de entrenamiento (2008 - 2014)
- Set de validación  (2015 - 2016)
- Set de prueba (2017-2018)



### Soluciones Propuestas
Se propone evaluar si utilizar una red multicapa en el entorno tensorflow para la implementación del proyecto (el cual se entrena con todo el dataset) o una red neuronal recurrente la cual posee una memoria 'selectiva' de partidos anteriores. Estas soluciones están sujetas a cambiar de acuerdo a las nuevas técnicas a aprender en la materia.

### Medición de la Calidad de la Solución Obtenida
La solución obtenida se limitará a predecir el ganador de cada partido. Esto tiene como consecuencia determinar el resultado de cada uno de los torneos a nivel ATP de la temporada, como así también el resultado del ranking al finalizar la temporada. Se contrastaran con los resultados reales, pretendiéndose como mínimo una efectividad superior al 50%.

### Proyectos Similares Implementados en Internet
El proyecto de referencia será la tesis presente en el siguiente link: https://www.doc.ic.ac.uk/teaching/distinguished-projects/2015/m.sipko.pdf el cuál plantea la predicción de partidos de tenis con diversos métodos incluyendo Machine Learning y mostrando sus resultados. Otro proyecto más breve se encuentra en el siguiente link: http://deepakn94.github.io/assets/papers/6.867.pdf. La técnica para solucionar el problema también puede referirse a predictores de otros deportes.
